# 기본옵션

In [2]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',60)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

# 경제

## 슈카월드 -465개

- 영상  정보 수집 ( 채널명/ 썸네일/ 구독자 수/ 제목 / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 / 영상길이 / 카테고리)
- 전체 영상 목록이 보이는 페이지에서 각 동영상의 url 추출하는 과정

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCsJ6RuBiTVWRX156FVbeaGg/videos') # ★★★ 변경 # 유뷰터 '동영상' 들어가서 전체 영상 목록 나오는 페이지 url
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림 # ★★★ 변경
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sukaworld_url = [] # ★★★ 이름 변경
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sukaworld_url.append(url) # ★★★ 이름 변경
    

In [ ]:
len(sukaworld_url)

- 추출한 모든 영상의 url을 입력하여, 각 페이지 안으로 들어가 정보를 수집
- 채널명/ 구독자 수 / 썸네일 / 제목 / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 / 영상길이 / 카테고리

In [ ]:
# ★★★ 이름 변경
sukaworld = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(sukaworld_url)):# ★★★ 이름 변경
    
    try :# 오휴생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sukaworld_url[i] # ★★★ 이름 변경
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        sukaworld = sukaworld.append(insert_data) # ★★★ 이름 변경
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
len(sukaworld)
sukaworld.head() # 이건 걍 무시해두됨 이미 저장하고 한번 더 돌리고 그냥 꺼버려서 그럼

In [ ]:
sukaworld.to_csv('sukaworld.csv',encoding='utf-8-sig')

## 신사임당 -334개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCaJdckl6MBdDPDf75Ec_bJA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 90 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sinsaimdang_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sinsaimdang_url.append(url)
    

In [ ]:
len(sinsaimdang_url)

In [ ]:
sinsaimdang = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(sinsaimdang_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sinsaimdang_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        sinsaimdang = sinsaimdang.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
sinsaimdang.shape
sinsaimdang.head()

In [ ]:
sinsaimdang.to_csv('sinsaimdang.csv', encoding='utf-8-sig')

## 돌디 -34개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UClmxSq4_kJfLtyIY7TycFfw/videos') 
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 10 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

doldi_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    doldi_url.append(url)
    

In [ ]:
len(doldi_url)

In [ ]:
doldi = pd.DataFrame({'channel_name' :[],
                       'subscribers' : [],
                       'thumbnail' : [],
                       'video_name':[],
                       'video_length':[],
                       'upload_date':[],
                       'hits' :[],
                       'likes_num':[],
                       'dislikes_num':[],
                       'category':[] })

for i in range(0,len(doldi_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    
        thumbnail_info = video_list[i].find('img', {'data-thumb' : True}) # src 없는 것들이 있어서 keyerror 'src' 피하려면, 있는 것들만
        if thumbnail_info != None :
            thumbnail = thumbnail_info['data-thumb'] 
        else : 
            thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = doldi_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text 
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                    'thumbnail' : [thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        doldi = doldi.append(insert_data)
    
    except AttributeError as d: 
        print(d)
    except IndexError as f:
        print(f)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
doldi.shape
doldi.head()

In [ ]:
doldi.to_csv('doldi.csv', encoding='utf-8-sig')

## 부동산 읽어주는 남자 -261개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UC2QeHNJFfuQWB4cy3M-745g/videos') 
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 50 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

budongsan_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    budongsan_url.append(url)
    

In [ ]:
len(budongsan_url)

In [ ]:
budongsan = pd.DataFrame({'channel_name' :[],
                       'subscribers' : [],
                       'thumbnail' : [],
                       'video_name':[],
                       'video_length':[],
                       'upload_date':[],
                       'hits' :[],
                       'likes_num':[],
                       'dislikes_num':[],
                       'category':[] })

for i in range(0,len(budongsan_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    
        thumbnail_info = video_list[i].find('img', {'data-thumb' : True}) # src 없는 것들이 있어서 keyerror 'src' 피하려면, 있는 것들만
        if thumbnail_info != None :
            thumbnail = thumbnail_info['data-thumb'] 
        else : 
            thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = budongsan_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text 
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                    'thumbnail' : [thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        budongsan = budongsan.append(insert_data)
    
    except AttributeError as d: 
        print(d)
    except IndexError as f:
        print(f)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
budongsan.shape
budongsan.head()

In [ ]:
budongsan.to_csv('budongsan.csv', encoding='utf-8-sig')

## 한국경제 TV -2226개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/user/hkwowtv/videos') 
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 300 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

hankook_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    hankook_url.append(url)
    

In [ ]:
len(hankook_url)

In [ ]:
hankook = pd.DataFrame({'channel_name' :[],
                       'subscribers' : [],
                       'thumbnail' : [],
                       'video_name':[],
                       'video_length':[],
                       'upload_date':[],
                       'hits' :[],
                       'likes_num':[],
                       'dislikes_num':[],
                       'category':[] })

for i in range(0,len(hankook_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    
        thumbnail_info = video_list[i].find('img', {'data-thumb' : True}) # src 없는 것들이 있어서 keyerror 'src' 피하려면, 있는 것들만
        if thumbnail_info != None :
            thumbnail = thumbnail_info['data-thumb'] 
        else : 
            thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = hankook_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text 
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                    'thumbnail' : [thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        hankook = hankook.append(insert_data)
    
    except AttributeError as d: 
        print(d)
    except IndexError as f:
        print(f)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
hankook.shape
hankook.head()

In [ ]:
hankook.to_csv('hankook.csv', encoding='utf-8-sig')

## 삼프로 TV -982개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UChlv4GSd7OQl3js-jkLOnFA/videos') 
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 200 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sampro_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sampro_url.append(url)
    

In [ ]:
len(sampro_url)

In [ ]:
sampro = pd.DataFrame({'channel_name' :[],
                       'subscribers' : [],
                       'thumbnail' : [],
                       'video_name':[],
                       'video_length':[],
                       'upload_date':[],
                       'hits' :[],
                       'likes_num':[],
                       'dislikes_num':[],
                       'category':[] })

for i in range(0,len(sampro_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    
        thumbnail_info = video_list[i].find('img', {'data-thumb' : True}) # src 없는 것들이 있어서 keyerror 'src' 피하려면, 있는 것들만
        if thumbnail_info != None :
            thumbnail = thumbnail_info['data-thumb'] 
        else : 
            thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sampro_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text 
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                    'thumbnail' : [thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        sampro = sampro.append(insert_data)
    
    except AttributeError as d: 
        print(d,i)
    except IndexError as f:
        print(f,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
sampro.shape
sampro.head()

In [ ]:
sampro.to_csv('sampro.csv', encoding='utf-8-sig')

## 황소장의 부동산 팩폭 -476개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UC2soFUK5L61ovrRW5L5u-JA/videos') 
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 200 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

hwang_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    hwang_url.append(url)
    

In [ ]:
len(hwang_url)

In [ ]:
hwang = pd.DataFrame({'channel_name' :[],
                       'subscribers' : [],
                       'thumbnail' : [],
                       'video_name':[],
                       'video_length':[],
                       'upload_date':[],
                       'hits' :[],
                       'likes_num':[],
                       'dislikes_num':[],
                       'category':[] })

for i in range(0,len(hwang_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    
        thumbnail_info = video_list[i].find('img', {'data-thumb' : True}) # src 없는 것들이 있어서 keyerror 'src' 피하려면, 있는 것들만
        if thumbnail_info != None :
            thumbnail = thumbnail_info['data-thumb'] 
        else : 
            thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = hwang_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text 
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'economy' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                    'thumbnail' : [thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        hwang = hwang.append(insert_data)
    
    except AttributeError as d: 
        print(d,i)
    except IndexError as f:
        print(f,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
hwang.shape
hwang.head()

In [ ]:
hwang.to_csv('hwang.csv', encoding='utf-8-sig')

# 요리 

## 승우아빠 -217개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCgsffS7MfKL6YU3r_U3E-aA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 50 # 승우아빠는 영상이 많아서 넉넉히 많이 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

seougwooappa_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    seougwooappa_url.append(url)

In [ ]:
len(seougwooappa_url)

In [ ]:
seougwooappa = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(seougwooappa_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = seougwooappa_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        seougwooappa = seougwooappa.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
seougwooappa.shape
seougwooappa.head()


In [ ]:
seougwooappa.to_csv('seougwooappa.csv', encoding='utf-8-sig')

## 요남 TV -32개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCyozK5OFN5lDrwim5wqQnLA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 5 # 5번 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.4)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

yonamtv_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    yonamtv_url.append(url)

In [ ]:
len(yonamtv_url)

In [ ]:
yonamtv = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(yonamtv_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = yonamtv_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        yonamtv = yonamtv.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
yonamtv.shape
yonamtv.head()

In [ ]:
yonamtv.to_csv('yonamtv.csv', encoding='utf-8-sig')

## 소소황 -170개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCWrh9haWsTEAQF9UEg6AEFQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 70
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sosohwang_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sosohwang_url.append(url)

In [ ]:
len(sosohwang_url)

In [ ]:
sosohwang = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(sosohwang_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sosohwang_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        sosohwang = sosohwang.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
sosohwang.shape
sosohwang.head()

In [ ]:
sosohwang.to_csv('sosohwang.csv', encoding='utf-8-sig')

## 심방골주부 -311개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCMXFSqYCTcB4iIJLwUJaBrQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 50
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

simbanggol_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    simbanggol_url.append(url)

In [ ]:
len(simbanggol_url)

In [ ]:
simbanggol = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(simbanggol_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = simbanggol_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        simbanggol = simbanggol.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
simbanggol.shape
simbanggol.head()

In [ ]:
simbanggol.to_csv('simbanggol.csv', encoding='utf-8-sig')

## 강쉪 -398개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCrxSR4OGbJBVjYxp4xjWgZQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 100
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

kangchef_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    kangchef_url.append(url)

In [ ]:
len(kangchef_url)

In [ ]:
kangchef = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(kangchef_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = kangchef_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        kangchef = kangchef.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
kangchef.shape
kangchef.head()

In [ ]:
kangchef.to_csv('kangchef.csv', encoding='utf-8-sig') 

## 만개의 레시피 -2257개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCKA_6r3CWC76x_EaFO6jsPA/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 300
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

mangaerecipe_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    mangaerecipe_url.append(url)

In [ ]:
len(mangaerecipe_url)

In [ ]:
mangaerecipe = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(mangaerecipe_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = mangaerecipe_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        mangaerecipe = mangaerecipe.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
mangaerecipe.shape
mangaerecipe.head()

In [ ]:
mangaerecipe.to_csv('mangaerecipe.csv',encoding='utf-8-sig')

## 산적 TV -248개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCdfhK0yIMjmhcQ3gP-qpXRw/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 150
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

sanjuck_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    sanjuck_url.append(url)

In [ ]:
len(sanjuck_url)

In [ ]:
sanjuck = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(sanjuck_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = sanjuck_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        sanjuck = sanjuck.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
sanjuck.shape
sanjuck.head()

In [ ]:
sanjuck.to_csv('sanjuck.csv',encoding='utf-8-sig')

## 최주부 -351개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCxGzGirbl7JZlTTrO7jJCLA/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 200
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

choijubu_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    choijubu_url.append(url)

In [ ]:
len(choijubu_url)

In [ ]:
choijubu = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(choijubu_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = choijubu_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        choijubu = choijubu.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
choijubu.shape
choijubu.head()

In [ ]:
choijubu.to_csv('choijubu.csv', encoding='utf-8-sig')

## 얌무 -478개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCO61Rl-jslhyeenfjt-NO4w/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 250
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

yammoo_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    yammoo_url.append(url)

In [ ]:
len(yammoo_url)

In [ ]:
yammoo = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(yammoo_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = yammoo_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        yammoo  = yammoo.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
yammoo.shape
yammoo.head()

In [ ]:
yammoo.to_csv('yammoo.csv', encoding='utf-8-sig')

## 복덩이 - 518개

In [ ]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('@needs_for_you/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UC4C7HXQgnuQPsk_d1HH4SHg/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 200
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

bokdunge_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    bokdunge_url.append(url)

In [ ]:
len(bokdunge_url)

In [ ]:
bokdunge = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits' :[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(bokdunge_url)):
    
    try : 
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src']        
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = bokdunge_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.4) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'cooking' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})

        bokdunge  = bokdunge.append(insert_data)

    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [ ]:
bokdunge.shape
bokdunge.head()

In [ ]:
bokdunge.to_csv('bokdunge.csv', encoding='utf-8-sig')

# 전체 파일 합치기

In [4]:
import glob
import os

In [5]:
input_file = r'C:\Users\LG\Desktop\딥러닝 과제_유튜브'
allfile_list = glob.glob(os.path.join(input_file,'*.csv'))
allfile_list

['C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\bokdunge.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\budongsan.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\choijubu.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\doldi.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\hankook.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\hwang.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\kangchef.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\mangaerecipe.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\sampro.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\sanjuck.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\seougwooappa.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\simbanggol.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\sinsaimdang.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\sosohwang.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\sukaworld.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\yammoo.csv',
 'C:\\Users\\LG\\Desktop\\딥러닝 과제_유튜브\\yonamtv.csv']

In [6]:
all_data=[]
for file in allfile_list:
    df=pd.read_csv(file,encoding='utf-8-sig')
    all_data.append(df)

In [8]:
data_combine = pd.concat(all_data, axis=0, ignore_index=True)

In [12]:
data_combine.shape
data_combine.head()

(9817, 11)

,Unnamed: 0,channel_name,subscribers,thumbnail,video_name,video_length,upload_date,hits,likes_num,dislikes_num,category
0,0,복덩이 - YouTube,구독자 10.4만명,https://i.ytimg.com/vi/al52wJgMGyI/hqdefault.jpg?sqp=-oa...,"너무나맛있는 '알타리묵은지찜과' ""앞치마에그림"" Altari old paper steamed ' An...",5:11,2020. 4. 11.,"1,126회",40개,1개,cooking
1,0,복덩이 - YouTube,구독자 10.4만명,https://i.ytimg.com/vi/-xx7shznKec/hqdefault.jpg?sqp=-oa...,헌옷으로만든 [앞치마]와 '민들레' 겉절이 'Apron' made with old clothes an...,6:56,2020. 4. 10.,"1,067회",85개,3개,cooking
2,0,복덩이 - YouTube,구독자 10.4만명,https://i.ytimg.com/vi/m7KC7oEKKuo/hqdefault.jpg?sqp=-oa...,탈모와다이어트에 탁월한고급진 '느타리버섯볶음' 과 프로방스 창문그리기 Stir-fried 'Oyste...,4:25,2020. 4. 9.,860회,64개,0개,cooking
3,0,복덩이 - YouTube,구독자 10.4만명,https://i.ytimg.com/vi/FYeuMGEgNWA/hqdefault.jpg?sqp=-oa...,감기 면연력 간기능보호에 탁월한 '도라지튀김' '민들레튀김' '라벤다그리기' Fried bellflo...,6:59,2020. 4. 8.,"1,116회",76개,1개,cooking
4,0,복덩이 - YouTube,구독자 10.4만명,https://i.ytimg.com/vi/UjLKW2fjkyA/hqdefault.jpg?sqp=-oa...,진달래화전 부침 복덩이표 바삭바삭한 쫀득쫀득비법공개,4:47,2020. 4. 7.,622회,47개,2개,cooking


In [13]:
data_combine.to_csv('cooking_economy.csv', encoding='utf-8-sig')